In [10]:
import sympy as sp
import sys
sys.path.append("../")
import pathlib as pl
from SymEigen import *
from sympy import symbols,sin,cos,Matrix
from project_dir import backend_source_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

In [11]:
k = Eigen.Scalar('kappa')
L0 = Eigen.Scalar('L0')

X = Eigen.Vector("X",6)
X0 = X[0:3,0]
X1 = X[3:6,0]
Cl = Gen.Closure(k, X, L0)
dist = sqrt((X0 - X1).T * (X0 - X1))
E = 0.5 * k * (dist-L0)**2
print("Energy:", E)

dEdX = VecDiff(E, X)
ddEddX = VecDiff(dEdX, X)
print("Gradient dE/dX:", dEdX)
print("Hessian d2E/dXdX:", ddEddX)

Energy: Matrix([[0.5*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))**2]])
Gradient dE/dX: Matrix([[1.0*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(0) - X(3))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)], [1.0*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(1) - X(4))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)], [1.0*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(2) - X(5))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)], [1.0*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(0) + X(3))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)], [1.0*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(1) + X(4))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)], [1.0*kappa*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(2) 

In [12]:
s = f'''
{Cl("dEdX",dEdX)}
{Cl("ddEddX",ddEddX)}
'''
print(s)

f = open( backend_source_dir('cuda') / 'inter_primitive_effect_system/constitutions/details/soft_vertex_stitch.inl', 'w')
f.write(s)
f.close()


template <typename T>
__host__ __device__ void dEdX(Eigen::Vector<T,6>& R, const T& kappa, const Eigen::Vector<T,6>& X, const T& L0)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
Symbol Name Mapping:
kappa:
    -> {}
    -> Matrix([[kappa]])
X:
    -> {}
    -> Matrix([[X(0)], [X(1)], [X(2)], [X(3)], [X(4)], [X(5)]])
L0:
    -> {}
    -> Matrix([[L0]])
*****************************************************************************************************************************/
/* Sub Exprs */
auto x0 = X(0) - X(3);
auto x1 = X(1) - X(4);
auto x2 = X(2) - X(5);
auto x3 = std::pow(std::pow(x0, 2) + std::pow(x1, 2) + std::pow(x2, 2), 1.0/2.0);
auto x4 = 1.0*ka